In [1]:
import pandas as pd

In [2]:
# SAMPLE_NAME="sample2"
# SAMPLE_NAME="sample3"
SAMPLE_NAME="sample4"

In [3]:
dataset_name = "NIKL_NEWSPAPER_2023_CSV"
fname = "NEWSPAPER_2022_1"
fname = "NEWSPAPER_2022_2"
fname = "NEWSPAPER_2022_3"

df = pd.read_parquet(f"source/{dataset_name}/{fname}.parquet")
print(df.shape, df.columns)

(114289, 11) Index(['file_id', 'doc_id', 'title', 'author', 'publisher', 'date', 'topic',
       'original_topic', 'sentence_ids', 'sentence_offsets', 'text'],
      dtype='object')


In [4]:
topics = df.topic.unique().tolist()
print(topics)

['생활', '정치', '사회', 'IT/과학', '경제', '미용/건강', '스포츠', '문화', '연예']


In [5]:
topic_ratio = {}

for k,v in df.topic.value_counts().to_dict().items():
    print(k,v)
    topic_ratio[k] = v/df.shape[0]

print(topic_ratio)

사회 31951
생활 21954
경제 17799
정치 17670
미용/건강 6420
스포츠 6378
IT/과학 6271
문화 5535
연예 311
{'사회': 0.27956321255763894, '생활': 0.19209197735565103, '경제': 0.15573677256778867, '정치': 0.15460805501841823, '미용/건강': 0.056173385015180814, '스포츠': 0.05580589558050206, 'IT/과학': 0.05486967249691571, '문화': 0.048429857641592806, '연예': 0.0027211717663117186}


In [6]:
# By-Ratio
total_n = 15000
per_topic_n = {
    k: int(total_n*v) for k,v in topic_ratio.items()
}
print(per_topic_n)

{'사회': 4193, '생활': 2881, '경제': 2336, '정치': 2319, '미용/건강': 842, '스포츠': 837, 'IT/과학': 823, '문화': 726, '연예': 40}


In [7]:
# # // n
# per_topic_n = {k: total_n//len(topic_ratio) for k in topic_ratio.keys()}
# print(per_topic_n)

In [8]:
sampled_df = []
for topic, count in per_topic_n.items():
    topic_df = df[df.topic==topic].sample(count)
    print(topic, topic_df.shape[0])
    
    sampled_df.append(topic_df)

sampled_df = pd.concat(sampled_df, axis=0)
print(sampled_df.shape)

사회 4193
생활 2881
경제 2336
정치 2319
미용/건강 842
스포츠 837
IT/과학 823
문화 726
연예 40
(14997, 11)


In [9]:
sampled_df.to_parquet(f"data/{dataset_name}-{fname}-{SAMPLE_NAME}.parquet")

In [10]:
sampled_df.columns

Index(['file_id', 'doc_id', 'title', 'author', 'publisher', 'date', 'topic',
       'original_topic', 'sentence_ids', 'sentence_offsets', 'text'],
      dtype='object')

In [11]:
metadata_df = sampled_df[['file_id', 'doc_id', 'topic', 'title']]
metadata_df.to_csv(f"data/{dataset_name}-{fname}-{SAMPLE_NAME}-metadata.tsv", sep='\t', index=None)